In [91]:
import logging

logging.basicConfig(
    level=logging.INFO,
    handlers=[logging.FileHandler(filename='Punteggi_prova.log')])
logger_prova = logging.getLogger('Risultati')

In [92]:
import pandas as pd
dataset = pd.read_excel('IncidentiModificato.xlsx', sheet_name='Foglio1', index_col=0)
dataset.head()

,DATA,SESSO,ANNI,PESO,ALTEZZA,BMI,Mezzo,Testa:Neurocranio,Testa:Splancnocranio,Testa:Telencefalo,...,Scheletro:Rachide-cervicale,Scheletro:Rachide-toracico,Scheletro:Rachide-lombare,Scheletro:Bacino-e-sacro,Scheletro:Complesso-sterno/claveo/costale,Tot Testa,Tot Torace,Tot Addome,Tot Scheletro,Totale
VERBALE,,,,,,,,,,,,,,,,,,,,,
85567,1999-10-29,0,81,84.0,1.75,27.428571,0,1,0,0,...,0,3,0,3,3,2,0,3,9,14
85829,2000-01-14,1,69,69.0,1.62,26.291724,1,4,4,4,...,0,0,0,0,4,20,7,1,4,32
85977,2000-03-10,1,71,67.0,1.55,27.887617,1,2,0,1,...,0,0,0,0,4,6,0,0,4,10
86220,2000-06-14,1,54,60.0,1.59,23.733238,1,4,0,0,...,0,0,0,0,4,5,3,2,4,14
86247,2000-06-22,1,78,69.0,1.67,24.740937,1,2,0,0,...,0,0,0,0,4,2,0,2,4,8


In [93]:
len(dataset)

130

In [94]:
dataset.index.unique()

Index([    85567,     85829,     85977,     86220,     86247,     86421,
           86839,     86876,     86878,     90056,
       ...
          101097,    101180,    101189,    101544,    101583,    101618,
       '101288E', 'E-97586',       'X',      'X1'],
      dtype='object', name='VERBALE', length=130)

In [95]:
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
import time
from random import seed
import numpy as np
import datetime
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from tqdm.notebook import tnrange

np.random.seed(42)

model_to_string_extended = {SVC: 'SVC', DecisionTreeClassifier: 'DecisionTreeClassifier', RandomForestClassifier:'RandomForestClassifier', GaussianNB: 'GaussianNB', LinearDiscriminantAnalysis: 'LinearDiscriminantAnalysis', MLPClassifier: 'MLPClassifier'}

def addestra_nuovo(model_class, X, y, model_selection_grid, num_fold_external, num_fold_internal, nome_dataset, logger_name=None, scaling=StandardScaler(), dim_reduction=None):
    
    ext_fold = StratifiedKFold(n_splits=num_fold_external, shuffle=True)
    test_errors = []
    for infer_indices, test_indices in ext_fold.split(X, y):
        print('start fold', end=' ')
        int_fold = StratifiedKFold(n_splits=num_fold_internal, shuffle=True)
        
        start_time = time.time()

        date_inizio = datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")
        if logger_name is not None: logger_name.info(f'INIZIO ADDESTRAMENTO alle {date_inizio}') 

        X_std = scaling.fit_transform(X)  if scaling is not None else X

        X_std = dim_reduction.fit_transform(X_std) if dim_reduction is not None else X_std
        
        clf = GridSearchCV(estimator=model_class(), param_grid=model_selection_grid, cv=int_fold, n_jobs=2)
        clf.fit(X_std[infer_indices], y.values[infer_indices])
        test_error = clf.best_estimator_.score(X_std[test_indices], y.values[test_indices])
        print('model used: {}'.format(model_to_string_extended[model_class]))
        print('best score: {:.3f}, best params: {}'.format(clf.best_score_, clf.best_params_), end=' ')
        print('test error: {:.3f} end fold'.format(test_error))
        test_errors.append(test_error)
        
        if logger_name is not None:
            logger_name.info(f'DATASET: {nome_dataset}')
            logger_name.info(f'MODEL: {model_to_string_extended[model_class]}')
            logger_name.info(f'SCALER: {scaling}')
            logger_name.info(f'RIDUZIONE DIMENSIONALITA {dim_reduction}')
            logger_name.info(f'IPERPARAMETRI: {clf.best_params_}')
            logger_name.info(f'TRAINING ERROR: {clf.best_score_}')
            logger_name.info(f'TEST ERROR: {np.mean(test_error)}')

        end_time = time.time()
        data_fine = datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")
        if logger_name is not None:
            logger_name.info(f'FINE ADDESTRAMENTO alle {data_fine}, tempo di calcolo totale in secondi: {end_time-start_time}')
    
        
        
    return np.mean(test_error)

In [96]:
X_total = dataset[['SESSO', 'ANNI', 'PESO', 'ALTEZZA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro']]
y_total = dataset['Mezzo']

In [97]:
models = [SVC, DecisionTreeClassifier, RandomForestClassifier, GaussianNB, LinearDiscriminantAnalysis, MLPClassifier]
model_to_string = {SVC: 'SVC', DecisionTreeClassifier: 'DT', RandomForestClassifier:'RF', GaussianNB: 'NB', LinearDiscriminantAnalysis: 'LD', MLPClassifier: 'MLP'}

c_space = np.logspace(-4, 3, 10)
gamma_space = np.logspace(-4, 3, 10)

model_selection_grid_SVC = [
  {'C': c_space, 'kernel': ['linear'], 'gamma': ['auto']},
  {'C': c_space, 'gamma': gamma_space, 'kernel': ['rbf']},
  {'C': c_space, 'gamma': ['auto', 'scale'], 'kernel': ['rbf']},
  {'C': c_space, 'degree': [2, 3, 5, 9], 'kernel': ['poly'], 'gamma': ['auto']},
 ]

model_selection_grid_DT = {'criterion': ['gini', 'entropy'],
                        'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                        'max_features': [None, 'sqrt', 'log2'],
                        'max_depth': [None, 2, 5, 10]}



model_selection_grid_RF = {'n_estimators': [5, 10, 50, 100, 200],
                        'criterion': ['gini', 'entropy'],
                        'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                        'max_features': [None, 'sqrt', 'log2'],
                        'max_depth': [None, 2, 5, 10]}

model_selection_grid_NB = {}
model_selection_grid_LD = {}

model_selection_grid_MLP = {'max_iter': [5000],
                        'hidden_layer_sizes': [[2], [4], [6], [10], [20], [4, 4], [10, 10]],
                        'activation': ['identity', 'logistic', 'tanh', 'relu']}

grids = [model_selection_grid_SVC, model_selection_grid_DT, model_selection_grid_RF, model_selection_grid_NB, model_selection_grid_LD, model_selection_grid_MLP]

In [98]:
import os.path
import json

def add_column_to_results_nuovo(result_dict, column_name, serialized_results_filename,
                          X, y, models, grids, scaling=StandardScaler(), dim_reduction=None, logger_name=logger_prova):
    
    if column_name not in result_dict:
        if os.path.exists(serialized_results_filename):
            with open(serialized_results_filename, 'r') as fp:
                saved_dict = json.load(fp)
                if column_name in saved_dict:
                    result_dict[column_name] = saved_dict[column_name]
                else:
                    result_dict[column_name] = {model_to_string[m]: np.mean(addestra_nuovo(m,
                                                                             X,
                                                                             y,
                                                                             g,
                                                                             5,
                                                                             3,
                                                                             scaling=scaling,
                                                                             dim_reduction=dim_reduction,
                                                                             nome_dataset=column_name,
                                                                             logger_name=logger_name))
                                                 for m, g in zip(models, grids)}
                    with open(serialized_results_filename, 'w', encoding='utf-8') as fp:
                        json.dump(result_dict, fp, ensure_ascii=False, indent=4)
        #Se il file json non esiste ancora non fa nulla, ho aggiunto allora un ramo else che lo crea per la prima volta
        else:
            result_dict[column_name] = {model_to_string[m]: np.mean(addestra_nuovo(m,
                                                                                 X,
                                                                                 y,
                                                                                 g,
                                                                                 5,
                                                                                 3,
                                                                                 scaling=scaling,
                                                                                 dim_reduction=dim_reduction,
                                                                                 nome_dataset=column_name,
                                                                                 logger_name=logger_name))
                                                 for m, g in zip(models, grids)}
            with open(serialized_results_filename, 'w', encoding='utf-8') as fp:
                json.dump(result_dict, fp, ensure_ascii=False, indent=4)

In [100]:
provaRisultatiStandardScaler = {}

In [101]:
add_column_to_results_nuovo(provaRisultatiStandardScaler, 'Totali', 'ProvaRisultatiStandardScaler.json', X_total, y_total, models, grids, logger_name=None)

start fold model used: SVC
best score: 0.721, best params: {'C': 27.825594022071257, 'gamma': 0.021544346900318846, 'kernel': 'rbf'} test error: 0.769 end fold
start fold model used: SVC
best score: 0.693, best params: {'C': 166.81005372000593, 'gamma': 0.003593813663804626, 'kernel': 'rbf'} test error: 0.654 end fold
start fold model used: SVC
best score: 0.692, best params: {'C': 27.825594022071257, 'gamma': 'auto', 'kernel': 'linear'} test error: 0.692 end fold
start fold model used: SVC
best score: 0.711, best params: {'C': 0.774263682681127, 'gamma': 0.1291549665014884, 'kernel': 'rbf'} test error: 0.615 end fold
start fold model used: SVC
best score: 0.654, best params: {'C': 0.774263682681127, 'gamma': 'auto', 'kernel': 'linear'} test error: 0.808 end fold
start fold model used: DecisionTreeClassifier
best score: 0.654, best params: {'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': 2} test error: 0.538 end fold
start fold model used: DecisionTreeCl

In [84]:
X_total_with_BMI = dataset[['SESSO', 'ANNI', 'PESO', 'ALTEZZA', 'BMI', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro']]
y_total_with_BMI = dataset['Mezzo']

In [85]:
add_column_to_results_nuovo(provaRisultatiStandardScaler, 'Totali_with_BMI', 'ProvaRisultatiStandardScaler.json', X_total_with_BMI, y_total_with_BMI, models, grids, logger_name=logger_prova)

start fold model used: SVC
best score: 0.692, best params: {'C': 0.774263682681127, 'gamma': 0.1291549665014884, 'kernel': 'rbf'} test error: 0.769 end fold
start fold model used: SVC
best score: 0.693, best params: {'C': 166.81005372000593, 'gamma': 0.003593813663804626, 'kernel': 'rbf'} test error: 0.692 end fold
start fold model used: SVC
best score: 0.683, best params: {'C': 166.81005372000593, 'gamma': 'auto', 'kernel': 'linear'} test error: 0.692 end fold
start fold model used: SVC
best score: 0.702, best params: {'C': 0.774263682681127, 'gamma': 'auto', 'kernel': 'rbf'} test error: 0.577 end fold
start fold model used: SVC
best score: 0.682, best params: {'C': 0.774263682681127, 'gamma': 'auto', 'kernel': 'linear'} test error: 0.808 end fold
start fold model used: DecisionTreeClassifier
best score: 0.683, best params: {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'log2', 'max_leaf_nodes': 5} test error: 0.538 end fold
start fold model used: DecisionTreeClassifier
bes

In [87]:
date_ordinate = dataset['DATA'].sort_values()
prima_data = date_ordinate.values[0]
print("La prima data del dataset è: ", prima_data)

La prima data del dataset è:  1999-10-29T00:00:00.000000000


In [88]:
import datetime as dt

dataset.DATA = dataset.DATA.apply(lambda d: (d - dt.datetime(1970,1,1)).days)

In [90]:
X_total_with_data = dataset[['DATA', 'SESSO', 'ANNI', 'PESO', 'ALTEZZA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro']]
y_total_with_data = dataset['Mezzo']

add_column_to_results_nuovo(provaRisultatiStandardScaler, 'Totali_with_DATA', 'ProvaRisultatiStandardScaler.json', X_total_with_data, y_total_with_data, models, grids, logger_name=logger_prova)

start fold model used: SVC
best score: 0.750, best params: {'C': 0.774263682681127, 'gamma': 0.1291549665014884, 'kernel': 'rbf'} test error: 0.692 end fold
start fold model used: SVC
best score: 0.730, best params: {'C': 4.641588833612782, 'gamma': 0.021544346900318846, 'kernel': 'rbf'} test error: 0.846 end fold
start fold model used: SVC
best score: 0.760, best params: {'C': 0.774263682681127, 'gamma': 0.021544346900318846, 'kernel': 'rbf'} test error: 0.769 end fold
start fold model used: SVC
best score: 0.711, best params: {'C': 4.641588833612782, 'gamma': 0.021544346900318846, 'kernel': 'rbf'} test error: 0.846 end fold
start fold model used: SVC
best score: 0.750, best params: {'C': 0.774263682681127, 'gamma': 'auto', 'kernel': 'rbf'} test error: 0.731 end fold
start fold model used: DecisionTreeClassifier
best score: 0.797, best params: {'criterion': 'gini', 'max_depth': 2, 'max_features': None, 'max_leaf_nodes': None} test error: 0.731 end fold
start fold model used: DecisionT